# EE2073 Lab 1
### Author: Nguyen Hoang Khanh (Kelvin)

Import required libraries and identify serial port

In [20]:
import serial
import serial.tools.list_ports

ports = serial.tools.list_ports.comports() # get list of serial ports with devices

Identify STM32 MCU

In [21]:
VID = 61525 # vendor ID of the device
PID = 38912 # product ID of the device

for p in ports:
    if p.vid == VID and p.pid == PID:
        try:
            device = serial.Serial(p.device)
        except serial.SerialException: # raised if the device not available
            print("Reconnect the controller unit.")

if device is None:
    raise Exception("No suitable device detected.") # if no matching device found

Call `device` object to retrieve basic information about the connection

In [22]:
device # check the connection information

Serial<id=0x23f4edd7760, open=True>(port='COM5', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)

Simple MicroPython commands to the controller unit

In [32]:
device.write(bytes('pyb.LED(1).toggle()\r', 'utf-8')) # toggle the status of D2 LED

20

If we run the line above after closing the connection, it will raise an error.

Close the connection

In [36]:
device.close() # close connection

Open the connection (to reopen after closing)

In [17]:
device.open() # reopen connection

Initialization for changing voltage at VDCP

In [33]:
device.write(bytes("from pyb import Pin\r", 'utf-8'))
device.write(bytes("from machine import SPI\r", 'utf-8'))
device.write(bytes("spi = SPI(sck=Pin('PB13', Pin.OUT), mosi=Pin('PB15', Pin.OUT), miso=Pin('PB14', Pin.IN))\r", 'utf-8'))
device.write(bytes("dz = Pin('PB12', Pin.OUT)\r", 'utf-8'))

26

Set the DC supply voltage at VDCP

In [35]:
expectedVolt = 12

device.write(bytes("volt={}\r".format(expectedVolt), 'utf-8'))
device.write(bytes("y=312-1020/volt\r", 'utf-8'))
device.write(bytes("dz.value(0)\r", 'utf-8'))
device.write(bytes("spi.write(b'\\x11')\r", 'utf-8'))
device.write(bytes("spi.write(bytes((int(y),)))\r", 'utf-8'))
device.write(bytes("dz.value(1)\r", 'utf-8'))

12

Here note that each line has to be written separately. We cannot write all lines at once using '\n' notation.

#### Explanation on SPI
SPI stands for Serial Peripheral Interface bus protocol. It is a synchronous serial protocol that is driven by a controller. By initiating a spi instance, we can communicate with a particular device on the bus.

The line `spi = SPI(sck=Pin('PB13', Pin.OUT), mosi=Pin('PB15', Pin.OUT), miso=Pin('Pin14', Pin.IN))` initiates a spi instance. It assigns the SCK to pin PB13 and sets the pin mode to output. Pin PB15 is selected to be the MOSI and set to output, while pin Pin14 is assigned to be the MISO and set to input.